In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
124423,Timothy Castagne,16,2,90,Leicester City,Newcastle United,1.207202,0.540499,True,5,...,0.0,0.0,0.0,32.0,38.0,84.2,26.0,0.0,0.0,1
124424,Timo Werner,16,4,73,Chelsea,Leeds,1.963548,0.569792,True,2,...,0.0,1.0,1.0,16.0,21.0,76.2,23.0,1.0,1.0,1
124425,Tim Iroegbunam,16,3,0,Aston Villa,Liverpool,0.909862,1.908485,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
124426,Tim Krul,16,1,90,Norwich,Manchester United,0.410660,2.007048,True,3,...,0.0,1.0,0.0,26.0,38.0,68.4,13.0,0.0,0.0,1
124427,Tino Livramento,16,2,90,Southampton,Arsenal,0.562634,1.241121,False,1,...,0.0,0.0,0.0,27.0,38.0,71.1,27.0,1.0,1.0,1
124428,Todd Cantwell,16,3,20,Norwich,Manchester United,0.410660,2.007048,True,1,...,0.0,0.0,0.0,7.0,7.0,100.0,7.0,1.0,2.0,1
124429,Tomas Soucek,16,3,90,West Ham United,Burnley,0.780243,0.320026,False,3,...,0.0,2.0,0.0,41.0,51.0,80.4,30.0,1.0,1.0,1
124430,Tommy Doyle,16,3,0,Manchester City,Wolverhampton Wanderers,2.378724,0.845989,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
124431,Tom Cleverley,16,3,57,Watford,Brentford,0.320467,0.438192,False,4,...,0.2,2.0,1.0,12.0,18.0,66.7,13.0,0.0,0.0,1
124432,Tom Davies,16,3,0,Everton,Crystal Palace,0.944331,0.549419,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

17

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
df_latest.head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Aaron Connolly,17,4,60,Brighton and Hove Albion,Wolverhampton Wanderers,0.565531,0.845216,True,2,...,9319,1,0,2.0,240,736,496,0,2021-12-15T19:30:00Z,2122
1,Aaron Cresswell,17,2,0,West Ham United,Arsenal,0.779531,1.239988,False,0,...,753488,0,2,0.0,-71817,1344,73161,0,2021-12-15T20:00:00Z,2122
2,Aaron Ramsdale,17,1,90,Arsenal,West Ham United,1.239988,0.779531,True,5,...,1459440,0,2,0.0,96475,114182,17707,1,2021-12-15T20:00:00Z,2122
3,Aaron Ramsey,17,3,0,Aston Villa,Norwich,0.909031,0.410285,False,0,...,17786,2,0,0.0,314,1219,905,0,2021-12-14T19:45:00Z,2122
4,Abdoulaye Doucouré,17,3,90,Everton,Chelsea,0.943469,1.961755,False,2,...,135411,1,1,4.0,-3605,2357,5962,0,2021-12-16T19:45:00Z,2122
5,Adama Traoré,17,3,85,Wolverhampton Wanderers,Brighton and Hove Albion,0.845216,0.565531,False,3,...,129733,1,0,2.0,-200,5069,5269,0,2021-12-15T19:30:00Z,2122
6,Adam Armstrong,17,4,0,Southampton,Crystal Palace,0.562120,0.548917,False,0,...,237402,2,2,0.0,-67553,1728,69281,0,2021-12-15T19:30:00Z,2122
7,Adam Forshaw,17,3,90,Leeds,Manchester City,0.569272,2.376551,False,2,...,3196,0,7,13.0,10,385,375,0,2021-12-14T20:00:00Z,2122
8,Adam Idah,17,4,29,Norwich,Aston Villa,0.410285,0.909031,True,1,...,38764,2,0,19.0,-480,1967,2447,0,2021-12-14T19:45:00Z,2122
9,Adam Lallana,17,3,6,Brighton and Hove Albion,Wolverhampton Wanderers,0.565531,0.845216,True,1,...,13844,1,0,2.0,-826,36,862,0,2021-12-15T19:30:00Z,2122


In [10]:
df_latest.shape

(460, 34)

In [11]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [12]:
last_play_proba.head()

,player,team,play_proba
0,Billy Gilmour,Norwich,1.0
1,Brandon Williams,Norwich,1.0
2,Michael McGovern,Norwich,1.0
3,Tim Krul,Norwich,1.0
4,Teemu Pukki,Norwich,1.0


In [13]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [14]:
df_latest.drop_duplicates(inplace=True)

In [15]:
df_train.drop_duplicates(inplace=True)

In [16]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [17]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [18]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [19]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Ederson
João Cancelo
Bernardo Silva
Rúben Dias
Rodri
Aymeric Laporte
Gabriel Jesus
Kyle Walker
Jack Grealish
İlkay Gündoğan
Phil Foden
Raheem Sterling
Kevin De Bruyne
Riyad Mahrez
Nathan Aké
John Stones
Ferrán Torres
Oleksandr Zinchenko
Fernandinho
Cole Palmer
Zack Steffen
Benjamin Mendy
James Mcatee
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Mohamed Salah
Sadio Mané
Alisson
Virgil van Dijk
Trent Alexander-Arnold
Jordan Henderson
Joël Matip
Andrew Robertson
Diogo Jota
Fabinho
Thiago Alcántara
Alex Oxlade-Chamberlain
Ibrahima Konaté
James Milner
Naby Keïta
Kostas Tsimikas
Roberto Firmino
Curtis Jones
Harvey Elliott
Caoimhín Kelleher
Takumi Minamino
Divock Origi
Joe Gomez
Neco Williams
Tyler Morton
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Edouard Mendy
Antonio Rüdiger
Jorginho
Thiago Silva
Marcos Alonso
Reece James
Ka

In [20]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [21]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [22]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [23]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [24]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4575,2021-2022,Jakob Sørensen,2021-12-05,Norwich,Premier League,15,0,0,10,4,...,0.0,0,0,7,9,77.8,9,1,1,Jacob Sørensen
4576,2021-2022,Jakob Sørensen,2021-12-11,Norwich,Premier League,16,1,0,49,14,...,0.0,0,0,36,41,87.8,24,0,0,Jacob Sørensen
4577,2021-2022,Jakob Sørensen,2021-12-14,Norwich,Premier League,17,2,1,68,28,...,0.0,0,0,42,45,93.3,35,0,0,Jacob Sørensen
4578,2021-2022,Sam Byram,2021-12-14,Norwich,Premier League,17,0,0,25,9,...,0.0,0,0,15,21,71.4,12,0,0,Sam Byram


In [25]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [26]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

[]

In [27]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [28]:
# any additional corrections
# match_list['Jakob Sørensen'] = 'Jacob Sørensen'
# match_list['Przemysław Płacheta'] = 'Przemyslaw Placheta'
# match_list['Mads Roerslev'] = 'Mads Roerslev Rasmussen'
# match_list['Odsonne Édouard'] = 'Odsonne Edouard'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

In [29]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [30]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

In [31]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [32]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [33]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [34]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [35]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [36]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [37]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [38]:
# add crowds field
df_latest_further['crowds'] = 1

In [39]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [40]:
df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds


In [41]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [42]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
120381,Harry Kane,10,4,90,Tottenham Hotspur,Manchester United,1.535124,2.007004,True,2,...,0.0,0.0,0.0,15.0,26.0,57.7,17.0,0.0,0.0,1
121004,Harry Kane,11,4,90,Tottenham Hotspur,Everton,1.535073,0.944279,False,2,...,0.3,3.0,0.0,26.0,35.0,74.3,32.0,1.0,2.0,1
121926,Harry Kane,12,4,90,Tottenham Hotspur,Leeds,1.535073,0.569761,True,2,...,0.0,2.0,1.0,20.0,24.0,83.3,32.0,3.0,6.0,1
122835,Harry Kane,14,4,90,Tottenham Hotspur,Brentford,1.535582,0.440358,True,2,...,0.5,6.0,1.0,22.0,31.0,71.0,31.0,2.0,3.0,1
123471,Harry Kane,15,4,90,Tottenham Hotspur,Norwich,1.535539,0.410615,True,2,...,0.1,2.0,1.0,13.0,21.0,61.9,22.0,2.0,4.0,1


In [43]:
df_train_new.tail(7)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
124926,Winston Reid,17,2,0,West Ham United,Arsenal,0.779531,1.239988,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
124927,Xherdan Shaqiri,17,3,0,Liverpool,Newcastle United,1.906742,0.540005,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
124928,Yan Valery,17,2,0,Southampton,Crystal Palace,0.562120,0.548917,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
124929,Yerry Mina,17,2,0,Everton,Chelsea,0.943469,1.961755,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
124930,Yerson Mosquera Valdelamar,17,2,0,Wolverhampton Wanderers,Brighton and Hove Albion,0.845216,0.565531,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
124931,Yves Bissouma,17,3,83,Brighton and Hove Albion,Wolverhampton Wanderers,0.565531,0.845216,True,1,...,0.0,1,0,51,56,91.1,36,1,1,1
124932,Zack Steffen,17,1,0,Manchester City,Leeds,2.376551,0.569272,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [44]:
df_latest_further.shape

(460, 57)

In [45]:
df_train_new.shape

(124933, 59)

In [46]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [47]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [48]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
177,Aston Villa,Burnley,2021-12-18T15:00:00Z,18.0,0
178,Leeds,Arsenal,2021-12-18T17:30:00Z,18.0,1
179,Newcastle United,Manchester City,2021-12-19T14:00:00Z,18.0,2
180,Wolverhampton Wanderers,Chelsea,2021-12-19T14:00:00Z,18.0,3
181,Tottenham Hotspur,Liverpool,2021-12-19T16:30:00Z,18.0,4
191,Newcastle United,Manchester United,2021-12-27T20:00:00Z,19.0,5
190,Brighton and Hove Albion,Brentford,2021-12-26T20:00:00Z,19.0,6
189,Aston Villa,Chelsea,2021-12-26T17:30:00Z,19.0,7
188,West Ham United,Southampton,2021-12-26T15:00:00Z,19.0,8
187,Tottenham Hotspur,Crystal Palace,2021-12-26T15:00:00Z,19.0,9


In [49]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [50]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,7,90
1,18.0,1,Leeds,Arsenal,2021-12-18T17:30:00Z,2,3
2,18.0,2,Newcastle United,Manchester City,2021-12-19T14:00:00Z,4,43
3,18.0,3,Wolverhampton Wanderers,Chelsea,2021-12-19T14:00:00Z,39,8
4,18.0,4,Tottenham Hotspur,Liverpool,2021-12-19T16:30:00Z,6,14


In [51]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [52]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
3,Pierre-Emerick Aubameyang,0,4,4,96,3,Aubameyang
8,Héctor Bellerín,0,2,9,48,3,Bellerín
10,Sead Kolasinac,0,2,11,43,3,Kolasinac
...,...,...,...,...,...,...,...
639,Owen Otasowie,0,3,442,45,39,Otasowie
641,Andreas Söndergaard,0,1,444,40,39,Söndergaard
643,Yerson Mosquera Valdelamar,0,2,446,45,39,Mosquera
648,Patrick Cutrone,0,4,523,50,39,Cutrone


In [53]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [54]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [55]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,18.0,0,Burnley,Aston Villa,2021-12-18T15:00:00Z,James Tarkowski,2,49,100,Tarkowski,False
1,18.0,0,Burnley,Aston Villa,2021-12-18T15:00:00Z,Phil Bardsley,2,39,100,Bardsley,False
2,18.0,0,Burnley,Aston Villa,2021-12-18T15:00:00Z,Erik Pieters,2,42,None,Pieters,False
3,18.0,0,Burnley,Aston Villa,2021-12-18T15:00:00Z,Jack Cork,3,44,None,Cork,False
4,18.0,0,Burnley,Aston Villa,2021-12-18T15:00:00Z,Dale Stephens,3,44,75,Stephens,False


In [56]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Jed Steer,1,40,None,Steer,True
1,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Conor Hourihane,3,49,0,Hourihane,True
2,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Emiliano Martínez,1,55,100,Martínez,True
3,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Björn Engels,2,45,0,Engels,True
4,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Bertrand Traoré,3,57,0,Traoré,True


In [57]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [58]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [59]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Jed Steer,1,4.0,1.0,Steer,True,0.909031,0.319734,2122
1,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Conor Hourihane,3,4.9,0.0,Hourihane,True,0.909031,0.319734,2122
2,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Emiliano Martínez,1,5.5,1.0,Martínez,True,0.909031,0.319734,2122
3,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Björn Engels,2,4.5,0.0,Engels,True,0.909031,0.319734,2122
4,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Bertrand Traoré,3,5.7,0.0,Traoré,True,0.909031,0.319734,2122
5,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Morgan Sanson,3,5.0,0.0,Sanson,True,0.909031,0.319734,2122
6,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,John McGinn,3,5.8,1.0,McGinn,True,0.909031,0.319734,2122
7,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Kortney Hause,2,4.4,1.0,Hause,True,0.909031,0.319734,2122
8,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Mahmoud Ahmed Ibrahim Hassan,3,5.4,1.0,Trézéguet,True,0.909031,0.319734,2122
9,18.0,0,Aston Villa,Burnley,2021-12-18T15:00:00Z,Tyrone Mings,2,5.0,1.0,Mings,True,0.909031,0.319734,2122


In [60]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')